In [1]:
import numpy as np

In [2]:
# 載入資料
X_train = np.load('X_train.npy')
X_val = np.load('X_val.npy')
X_test = np.load('X_test.npy')
y_train = np.load('y_train.npy')
y_val = np.load('y_val.npy')
y_test = np.load('y_test.npy')

In [3]:
# 重塑數據形狀為 (樣本數, 高度, 寬度, 頻道數)
X_train = X_train[..., np.newaxis]
X_val = X_val[..., np.newaxis]
X_test = X_test[..., np.newaxis]

print(X_train.shape)
print(X_val.shape)
print(X_test.shape)

(386, 64, 1034, 1)
(48, 64, 1034, 1)
(49, 64, 1034, 1)


In [4]:
import tensorflow as tf
from keras import Input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization

def create_model(input_shape):
    input_img = Input(shape=input_shape)
    x = Conv2D(32, kernel_size=(3, 3), activation='relu', padding='same')(input_img)
    x = BatchNormalization()(x)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Conv2D(128, kernel_size=(3, 3), activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Conv2D(256, kernel_size=(3, 3), activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Conv2D(512, kernel_size=(3, 3), activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Flatten()(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.2)(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.2)(x)
    x = Dense(64, activation='relu')(x)
    x = Dropout(0.2)(x)
    output = Dense(1, activation='sigmoid')(x)
    model = Model(inputs=input_img, outputs=output)
    return model


In [5]:
input_shape = np.shape(X_train)[1:]
model = create_model(input_shape)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 64, 1034, 1)]     0         
                                                                 
 conv2d (Conv2D)             (None, 64, 1034, 32)      320       
                                                                 
 batch_normalization (BatchN  (None, 64, 1034, 32)     128       
 ormalization)                                                   
                                                                 
 max_pooling2d (MaxPooling2D  (None, 32, 517, 32)      0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 32, 517, 64)       18496     
                                                                 
 batch_normalization_1 (Batc  (None, 32, 517, 64)      256   

# 編譯模型

In [6]:
from keras.optimizers import Adam

from tensorflow.keras.callbacks import ReduceLROnPlateau

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=0.00001)
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])
history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=100, batch_size=32, callbacks=[reduce_lr], verbose=2)


Epoch 1/100
13/13 - 13s - loss: 2.5397 - accuracy: 0.5518 - val_loss: 0.7354 - val_accuracy: 0.5208 - lr: 0.0010 - 13s/epoch - 1s/step
Epoch 2/100
13/13 - 1s - loss: 3.0418 - accuracy: 0.5104 - val_loss: 1.1550 - val_accuracy: 0.5208 - lr: 0.0010 - 1s/epoch - 103ms/step
Epoch 3/100
13/13 - 1s - loss: 2.4537 - accuracy: 0.4845 - val_loss: 0.7099 - val_accuracy: 0.5208 - lr: 0.0010 - 1s/epoch - 102ms/step
Epoch 4/100
13/13 - 1s - loss: 2.1851 - accuracy: 0.4689 - val_loss: 0.7385 - val_accuracy: 0.5208 - lr: 0.0010 - 1s/epoch - 103ms/step
Epoch 5/100
13/13 - 1s - loss: 1.9801 - accuracy: 0.5130 - val_loss: 0.7691 - val_accuracy: 0.5208 - lr: 0.0010 - 1s/epoch - 102ms/step
Epoch 6/100
13/13 - 1s - loss: 1.7370 - accuracy: 0.5337 - val_loss: 0.6975 - val_accuracy: 0.5000 - lr: 0.0010 - 1s/epoch - 102ms/step
Epoch 7/100
13/13 - 1s - loss: 1.6633 - accuracy: 0.4974 - val_loss: 0.6874 - val_accuracy: 0.5625 - lr: 0.0010 - 1s/epoch - 103ms/step
Epoch 8/100
13/13 - 1s - loss: 1.3739 - accuracy: